---
title: Multi-Modal Tutorial Dataset 1
numbering:
  enumerator: 4.%s
jupytext:
  formats: md:myst
  text_representation:
    extension: .md
    format_name: myst
kernelspec:
  display_name: Python 3
  language: python
  name: python3
---

### Guided Computation of Fused Multi-Modal Electron Microscopy

+++ {"part": "Code Walkthrough"} 
In this tutorial we walkthough how you can fuse your EELS/EDX maps with HAADF or similar elastic imaging modalities to improve chemical resolution. This is Tutorial 1 of 2 where we look at an atomic resolution HAADF and EDX dataset of DyScO$_3$. The multi-modal data fusion workflow relies on Python, and requires minimal user input with <10 tunable lines. Both here and in the Mathematical Overview section we outline best practices for these adjustments.  Within a few minutes, datasets such as the one in this tutorial can be transformed into resolution enhanced chemical maps. (Figure 4.1)
+++

:::{figure} ./figs/Figure_3_Output.png
:name: Raw vs Fused DyScO$_3$
:width: 700px
Comparison of raw input vs fused multi-modal DyScO$_3$ HAADF elastic and EDX inelastic images
:::
```{warning} Step 0: Experimental Requirements
To reconstruct using fused multi-modal electron microscopy you need to collect both elastic (e.g. HAADF) and inelastic (e.g. EELS / EDX) maps of your material. For the elastic signal, it is important that it provides Z-contrast of your elements. For the inelastic signal, you should have all chemistries in your sample mapped. Solving for under-determined chemical maps, or using difficult to interpret elastic signals are outside the scope of this tutorial. The collected chemical maps and HAADF must all have the same dimensionality, i.e. the same image size and number of pixels.  For this reason, we recommend using the HAADF signal that is simultaneously collected when taking an EDX/EELS scan
```

```{admonition} Step 1: Python Imports
First, we import standard python packages alongside our custom class of functions for data fusion called [fusion_utils](https://github.com/jtschwar/Multi-Modal-2D-Data-Fusion/blob/170fea3292da7e6390bfff7236610eb0c8077ff7/EDX/fusion_utils.py). The fusion_utils package contains 3 functions and a class of TV functions: save_data(), plot_convergence(), create_weighted_measurement_matrix(), and tvlib(). The save_data function saves the fused images into .tif files and all the parameters and matrix data into a .h5 file. Plot_convergence plots the convergence of the cost functions. Create_weighted_measurement_matrix is for generating the first part of our cost function that relates the elastic and inelastic modalities. Finally, the class of functions called tvlib performs the Fast Gradient Project (FGP) method of TV image denoising, which deblurs output images while preserving edges.
```

In [11]:
import data.fusion_utils as utils

from scipy.sparse import spdiags
import matplotlib.pyplot as plt
from tqdm import tqdm 
import numpy as np

ModuleNotFoundError: No module named 'skimage'

```{admonition} Step 2: Load your data
Load your inelastic and elastic data. Define the element names and their corresponding atomic weights. For the sake of this tutorial a generic .npz file is used, but for .dm3,.dm4,.emd, or another EM file format, just extract the 2D image data into numpy matrices. In Tutorial 2 this is shown by extracting from a .h5 file. Your elastic data should be stored in the HAADF variable, and your inelastic data (EDX/EELS) should be stored in the chemMap variable.
```
```{tip} Loading alternate file formats
If you are loading data from a .dm3, .dm4 or .emd file, we recommend you use HyperSpy.  The documentation for loading and saving data from those file types can be found [here](https://hyperspy.org/hyperspy-doc/v1.3/user_guide/io.html).
```